In [1]:
import numpy as np
import pandas as pd

from astropy.io import fits

#fits_image_filename = fits.util.get_testdata_filepath()

In [2]:
_epsilon = 3e-4 

In [3]:
hdul_gev = fits.open('gll_psc_v16.fit')
cat_gev = hdul_gev[1].data

In [4]:
names_gev = [
    'ASSOC_TEV', 
    'CLASS1', 
    'GLON', 
    'GLAT', 
    'Variability_Index', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000',  
    'Flux30_100',
    ] 

In [5]:
hdul_tev = fits.open('gammacat.fits.gz')
cat_tev = hdul_tev[1].data

In [6]:
names_tev = [
    'classes', 
    'glat', 
    'glon', 
    'morph_pa', 
    'sed_dnde', 
    'sed_dnde_err', 
    'sed_e_ref', 
    'spec_dnde_1TeV', 
    'spec_dnde_1TeV_err', 
    'spec_eflux_1TeV_10TeV', 
    'spec_eflux_1TeV_10TeV_err', 
    'spec_flux_1TeV', 
    'spec_flux_1TeV_crab', 
    'spec_flux_1TeV_crab_err'
    ]

In [7]:
def corresponding_bins(cat_gev, cat_tev, epsilon):
    d = {}
    
    class_gev = cat_gev['CLASS1']
    class_tev = cat_tev['classes']
    glat_gev = cat_gev['GLAT']
    glat_tev = cat_tev['glat']
    glon_gev = cat_gev['GLON']
    glon_tev = cat_tev['glon']
    
    for j in range(len(glat_tev)):
        for i in range(len(glat_gev)):
            if ((class_tev[j].find('bin') != -1) 
                & ((class_gev[i].find('BIN') != -1) | (class_gev[i].find('HMB') != -1))
                & (np.abs(glat_gev[i] - glat_tev[j]) < epsilon) 
                & (np.abs(glon_gev[i] - glon_tev[j]) < epsilon)):
                d.update({j : i})
    return d
                
print(corresponding_bins(cat_gev, cat_tev, 1))

{125: 2358, 13: 319, 45: 1172}


In [8]:
gevToTev = {'BLL': 'blazar', 
            'FRSQ': 'frsq', 
            'HMB': 'bin' , 
            'BIN': 'bin', 
            'GAL': 'galaxy', 
            'PSR': 'psr', 
            'PWN': 'pwn', 
            'SNR': 'snr', 
            '': 'unid'}
tevToGev = {v:k for k, v in gevToTev.items()}

In [9]:
def common(cat_gev, cat_tev, epsilon):
    """This function looks for the same objects in GeV and TeV catalogs
    
    Return: dictionary with corresponding values
    
    feature1(list) - features of GeV 
    feature2(list) - features of TeV 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}

    class_gev = cat_gev['CLASS1']
    class_tev = cat_tev['classes']
    glat_gev = cat_gev['GLAT']
    glat_tev = cat_tev['glat']
    glon_gev = cat_gev['GLON']
    glon_tev = cat_tev['glon']
    
    for i in range(len(glat_gev)):
        for j in range(len(glat_tev)):
            classGeV = class_gev[i]
            start = -1
            try: 
                start = (class_tev[j].find(gevToTev[classGeV]))
            except KeyError:
                continue
            if (start != -1):
                if ((glat_gev[i] - glat_tev[j] < epsilon) and (glon_gev[i] - glon_tev[j] < epsilon)) :
                    d.update({j : i})
    return d

D = common(cat_gev, cat_tev, 3e-10)
print(D)

{0: 2699, 2: 2699, 8: 2405, 9: 2884, 11: 3030, 13: 2358, 24: 2994, 29: 2699, 31: 2994, 37: 620, 39: 2699, 41: 2884, 44: 955, 46: 3030, 47: 955, 50: 955, 60: 2358, 61: 2354, 64: 955, 65: 2405, 66: 2405, 68: 3030, 70: 2699, 73: 2699, 74: 955, 75: 2699, 76: 3030, 77: 3030, 79: 1847, 81: 2699, 82: 1847, 84: 3030, 85: 2405, 86: 3030, 87: 2405, 88: 3030, 89: 2405, 90: 3030, 91: 3030, 93: 3030, 94: 955, 95: 3030, 96: 2699, 97: 2699, 98: 2699, 99: 2699, 100: 1847, 103: 3030, 104: 2699, 105: 3030, 107: 3030, 108: 3030, 109: 3030, 111: 2237, 113: 2704, 115: 2274, 117: 2926, 118: 2926, 119: 2926, 120: 2926, 122: 2286, 123: 2926, 126: 2926, 127: 2926, 128: 2926, 129: 2926, 131: 2926, 132: 2354, 133: 2926, 134: 2926, 135: 2926, 136: 2405, 137: 2926, 138: 2405, 139: 2926, 140: 2926, 141: 2926, 142: 2984, 143: 2440, 144: 2984, 146: 2499, 151: 2699, 152: 3030, 153: 3030, 154: 3030, 155: 3030, 156: 3030, 159: 2699, 160: 2699, 163: 2699}


In [10]:
names_common = [
    'glat',
    'glon',
    'morph_pa',
    'sed_dnde',
    'sed_dnde_err',
    'sed_e_ref', 
    'spec_dnde_1TeV', 
    'spec_dnde_1TeV_err', 
    'spec_eflux_1TeV_10TeV', 
    'spec_eflux_1TeV_10TeV_err', 
    'spec_flux_1TeV', 
    'spec_flux_1TeV_crab', 
    'spec_flux_1TeV_crab_err', 
    'ASSOC_TEV', 
    'Variability_Index', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000', 
    'Flux30_100', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000', 
    'Flux30_100', 
    'CLASS1']

In [11]:
d_coord = {'glon' : 'GLON', 'glat' : 'GLAT'}

In [12]:
def create_common_data(cat_gev, cat_tev, D, namefinal):
    """The fonction adds objects found both in GeV and TeV.
    
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    namefinal(list) - names of required features to fill in data
    """
    k = 0
    data = []
    
    for j in D.keys():
        data.append([])
        for i in range(13):
            data[k].append(cat_gev[namefinal[i]][j])
        for i in range(13, len(namefinal)):
            data[k].append(cat_tev[namefinal[i]][D[j]])
        k = k+1
        
    df_common = pd.DataFrame(data = data, columns = namefinal)
        
    return df_common
    #print(str(cat['CLASS1'][D[i]]) + " - " + str(cat2['classes'][i]) + "\n")

In [13]:
def create_only_tev_data(cat_tev, D, name_tev):
    """The fonction adds objects found only in TeV.
    
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    name_tev(list) - names of TeV columns
    """
    k = 0
    data = []
    
    for j in range(len(cat_tev)):
        if not(j in D.keys()):
            data.append([])
            for i in range(len(name_tev)):
                data[k].append(cat_tev[name_tev[i]][j])
            k = k+1
            
    df_only_tev = pd.DataFrame(data = data, columns = name_tev)
        
    return df_only_tev

In [14]:
def create_only_gev_data(cat_gev, D, names_gev):
    """The fonction adds objects found only in GeV.
    
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    name_gev(list) - names of GeV columns
    """
    k = 0
    data = []
    
    for j in range(len(cat_gev)):
        if not(j in D.values()):
            data.append([])
            for i in range(len(names_gev)):
                data[k].append(cat_gev[names_gev[i]][j])
            k = k+1
            
    df_only_gev = pd.DataFrame(data = data, columns = names_gev)
        
    return df_only_gev

In [15]:
data = []
common_data = create_common_data(cat_tev, cat_gev, D, names_common)
print("Added common values = " + str(len(common_data)))
only_tev_data = create_only_tev_data(cat_tev, D, names_tev)
print("Added values which can be found only in grammacat.fits (TeV) = " + str(len(only_tev_data)))
only_gev_data = create_only_gev_data(cat_gev, D, names_gev)
print("Added values which can be found only in gll_psc_v16.fit (GeV) = " + str(len(only_gev_data)))

print("Size of grammacat.fits (TeV) = "  + str(len(cat_tev)))
print("Size of gll_psc_v16.fits (GeV) = "  + str(len(cat_gev)))

Added common values = 90
Added values which can be found only in grammacat.fits (TeV) = 76
Added values which can be found only in gll_psc_v16.fit (GeV) = 3016
Size of grammacat.fits (TeV) = 166
Size of gll_psc_v16.fits (GeV) = 3034


In [16]:
common_data.head()

,glat,glon,morph_pa,sed_dnde,sed_dnde_err,sed_e_ref,spec_dnde_1TeV,spec_dnde_1TeV_err,spec_eflux_1TeV_10TeV,spec_eflux_1TeV_10TeV_err,...,Flux300_1000,Flux30_100,Flux1000,Flux10000_100000,Flux1000_3000,Flux100_300,Flux3000_10000,Flux300_1000,Flux30_100,CLASS1
0,10.203682,119.580254,17.4,"[1.9e-12, 7.3e-13, 1.2e-13, 3.4e-14, 1.2e-14, ...","[9e-13, 1.7e-13, 4e-14, 1.2e-14, 5e-15, 2.5e-1...","[0.78, 1.39, 2.47, 4.39, 7.81, 13.89, nan, nan...",1.020254e-12,2.673885e-13,3.016235e-12,4.314187e-13,...,3.461372e-08,NaN,1.021441e-08,1.195312e-10,8.466375e-09,5.964314e-08,1.694468e-09,3.461372e-08,NaN,SNR
1,1.412930,120.092361,NaN,"[1.83e-12, 2.29e-13, 4.56e-14, 6.62e-15, 6.05e...","[6.48e-13, 9.1e-14, 2.13e-14, 4.9e-15, 2.44e-1...","[0.5242, 0.9323, 1.658, 2.948, 5.242, 9.4, nan...",2.200000e-13,5.000000e-14,3.370669e-13,1.334992e-13,...,3.461372e-08,NaN,1.021441e-08,1.195312e-10,8.466375e-09,5.964314e-08,1.694468e-09,3.461372e-08,NaN,SNR
2,3.083848,130.717819,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",2.000000e-13,4.000000e-14,4.821693e-13,1.348196e-13,...,1.076956e-08,NaN,1.184660e-08,1.087441e-09,8.559550e-09,5.170882e-09,3.236763e-09,1.076956e-08,NaN,PWN
3,-23.486858,142.601715,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,NaN,...,2.061498e-08,NaN,6.859273e-09,2.632098e-10,5.606026e-09,5.109028e-08,1.246978e-09,2.061498e-08,NaN,BLL
4,-16.766897,140.142960,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,NaN,...,1.920689e-10,NaN,3.016442e-10,3.124023e-11,1.822992e-10,1.477740e-09,1.212272e-10,1.920689e-10,NaN,


In [17]:
only_tev_data.head()

,classes,glat,glon,morph_pa,sed_dnde,sed_dnde_err,sed_e_ref,spec_dnde_1TeV,spec_dnde_1TeV_err,spec_eflux_1TeV_10TeV,spec_eflux_1TeV_10TeV_err,spec_flux_1TeV,spec_flux_1TeV_crab,spec_flux_1TeV_crab_err
0,hbl,-78.086937,74.632011,NaN,"[1.907e-12, 1.462e-13, 8.40099e-15, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.4452, 0.949698, 2.121, nan, nan, nan, nan, ...",1.175430e-13,4.592630e-14,1.291623e-13,8.729060e-14,4.897623e-14,0.236093,0.136923
1,hbl,-81.216103,94.174644,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,hbl,-2.978120,120.975426,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,galaxy,-87.964546,97.363853,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hbl,-22.939846,132.416458,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
only_gev_data.head()

,ASSOC_TEV,CLASS1,GLON,GLAT,Variability_Index,Flux1000,Flux10000_100000,Flux1000_3000,Flux100_300,Flux3000_10000,Flux300_1000,Flux30_100
0,,,117.693878,3.402958,40.753517,1.015962e-09,2.839629e-11,1.236649e-09,1.808316e-08,5.780520e-11,6.940733e-09,NaN
1,,,345.410522,-74.946754,44.845318,2.038660e-10,1.590734e-11,1.450903e-10,9.876651e-11,7.452389e-11,1.342487e-10,NaN
2,,spp,117.293091,0.925701,53.047859,6.403683e-10,2.021964e-12,3.020877e-10,2.913030e-08,1.158661e-10,4.650789e-09,NaN
3,,bll,89.022202,-67.324249,49.738213,6.952190e-10,1.005633e-11,5.223298e-10,4.086941e-09,2.106892e-10,2.083574e-09,NaN
4,,fsrq,107.665428,-40.047157,130.336334,2.944868e-10,1.035538e-14,3.565206e-10,1.523905e-08,8.413887e-15,3.639644e-09,NaN
